In [1]:
# Function to load counts from CSV
def load_csv_counts(filename):
    counts = {}
    with open(filename, "r", encoding="utf-8") as f:
        header = f.readline()
        for line in f:
            parts = line.strip().split(",")
            if len(parts) < 2:
                continue
            ngram_str = parts[0].strip().strip('"')
            count_str = parts[1].strip().strip('"')
            try:
                count = int(count_str)
            except:
                continue
            ngram = tuple(ngram_str.split())
            if len(ngram) == 0:
                continue
            counts[ngram] = count
    return counts

# Load trigram and quadrigram
tri_counts  = load_csv_counts(r"C:\Users\evilk\OneDrive\Desktop\III YEAR\LABS\NLP\LAB4\trigram.csv")
quad_counts = load_csv_counts(r"C:\Users\evilk\OneDrive\Desktop\III YEAR\LABS\NLP\LAB4\quadrigram.csv")

ngram_counts = {3: tri_counts, 4: quad_counts}


In [2]:
import random

def get_prob(word, context, counts_dicts, n):
    """
    Returns MLE probability of 'word' given 'context'.
    Uses backoff if context not found.
    """
    if n == 1:
        total = sum(counts_dicts[3].values())  # fallback: use trigram counts last word frequencies
        word_counts = sum(c for g, c in counts_dicts[3].items() if g[-1]==word)
        return word_counts / total if total > 0 else 0

    ctx = tuple(context[-(n-1):])
    ngram = ctx + (word,)
    c_ngram = counts_dicts[n].get(ngram, 0)
    c_prefix = sum([c for g, c in counts_dicts[n].items() if g[:-1] == ctx])
    if c_prefix == 0:
        return get_prob(word, context, counts_dicts, n-1)
    return c_ngram / c_prefix


In [3]:
def get_start_context(counts_dicts, n):
    # Choose a context that exists in the n-grams and starts with <s>
    candidates = [g[:-1] for g in counts_dicts[n] if g[0] == "<s>"]
    if candidates:
        return list(random.choice(candidates))
    else:
        return ["<s>"] * (n-1)


In [ ]:
def generate_greedy_ng(counts_dicts, n, max_len=15):
    sentence = get_start_context(counts_dicts, n)
    
    for _ in range(max_len):
        ctx = tuple(sentence[-(n-1):])
        candidates_dict = {g[-1]: counts_dicts[n][g] for g in counts_dicts[n] if g[:-1]==ctx}
        
        if not candidates_dict:
            # fallback: pick any last word from trigram
            last_words = [g[-1] for g in counts_dicts[3]]
            next_word = random.choice(last_words)
            sentence.append(next_word)
            if next_word == "</s>":
                break 
            continue
        
        # Probabilistic sampling
        words, counts = zip(*candidates_dict.items())
        probs = [c/sum(counts) for c in counts]
        next_word = random.choices(words, probs)[0]
        sentence.append(next_word)
        if next_word == "</s>":
            break
    return " ".join(sentence) 


In [5]:
def generate_beam_ng(counts_dicts, n, beam_size=20, max_len=15):
    sequences = [(get_start_context(counts_dicts, n), 1.0)]
    
    for _ in range(max_len):
        all_candidates = []
        for seq, seq_prob in sequences:
            ctx = tuple(seq[-(n-1):])
            candidates_dict = {g[-1]: counts_dicts[n][g] for g in counts_dicts[n] if g[:-1]==ctx}
            
            if not candidates_dict:
                # fallback: pick any last word from trigram
                last_words = [g[-1] for g in counts_dicts[3]]
                next_word = random.choice(last_words)
                all_candidates.append((seq+[next_word], seq_prob))
                continue
            
            words, counts = zip(*candidates_dict.items())
            probs = [c/sum(counts) for c in counts]
            for w, p in zip(words, probs):
                all_candidates.append((seq+[w], seq_prob * p))
        
        all_candidates.sort(key=lambda x: x[1], reverse=True)
        sequences = all_candidates[:beam_size]
        
        if all(seq[-1] == "</s>" for seq, _ in sequences):
            break
    return [" ".join(seq) for seq, _ in sequences]


In [6]:
import csv

n_values = [3,4]  # trigram and quadrigram
num_sentences = 100

for n in n_values:
    # Greedy
    greedy_sentences = [generate_greedy_ng(ngram_counts, n) for _ in range(num_sentences)]
    output_file = f"C:\\Users\\evilk\\OneDrive\\Desktop\\III YEAR\\LABS\\NLP\\LAB6\\greedy_{n}gram_100.csv"
    with open(output_file, "w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Sentence"])
        for s in greedy_sentences:
            writer.writerow([s])
    
    # Beam search
    beam_sentences = []
    while len(beam_sentences) < num_sentences:
        seqs = generate_beam_ng(ngram_counts, n, beam_size=20)
        beam_sentences.extend(seqs)
    beam_sentences = beam_sentences[:num_sentences]
    
    output_file_beam = f"C:\\Users\\evilk\\OneDrive\\Desktop\\III YEAR\\LABS\\NLP\\LAB6\\beam_{n}gram_100.csv"
    with open(output_file_beam, "w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Sentence"])
        for s in beam_sentences:
            writer.writerow([s])
    
    print(f"✅ {n}-gram greedy & beam sentences saved.")


✅ 3-gram greedy & beam sentences saved.
✅ 4-gram greedy & beam sentences saved.


Beam Search is almost always better than Greedy Search for generating high-quality, coherent sentences.
Greedy search is faster and simpler, but it often leads to repetitive or nonsensical output because it makes short-sighted decisions. Beam search is a compromise that produces far more fluent and logical sentences by keeping its options open.

<!-- Let's prove with a simple example how Greedy Search can fail where Beam Search succeeds.
Scenario: We have a bigram model and want to generate a sentence.
Vocabulary: <s>, I, am, happy, Sam, </s>
Beam Width (k): 2
Our Model's Probabilities (designed to trap Greedy Search):
| P(w₂ | w₁) | Probability | Note |
| :--- | :--- | :--- |
| P(I | <s>) | 1.0 | The sentence must start with "I" |
| P(am | I) | 0.8 | Looks like the best first step |
| P(happy | I) | 0.2 | A less likely, but possible first step |
| P(Sam | am) | 0.1 | The path after "am" leads to a low-probability word |
| P(am | happy) | 0.9 | The path after "happy" leads to a very high-probability word |
| P(</s> | Sam) | 1.0 | The only way to end the "Sam" sentence |
| P(</s> | am) | 1.0 | The only way to end the "am" sentence (after "happy") |
1. Greedy Search Walkthrough
Start: The sequence is <s>.
Step 1: The only possible next word is I.
Sequence: I. Probability: P(I|<s>) = 1.0.
Step 2 (The Greedy Decision):
The model compares P(am | I) = 0.8 with P(happy | I) = 0.2.
0.8 > 0.2, so the greedy choice is am. It permanently discards the "happy" path.
Sequence: I am. Cumulative Probability: 1.0 * 0.8 = 0.8.
Step 3: From "am", the only option is "Sam".
Sequence: I am Sam. Cumulative Probability: 0.8 * P(Sam | am) = 0.8 * 0.1 = 0.08.
Step 4: From "Sam", the only option is </s>.
Final Sequence: I am Sam </s>. Final Probability: 0.08.
Result: Greedy Search generates "I am Sam" with a probability of 0.08.
2. Beam Search Walkthrough (Beam Width = 2)
Start: The sequence is <s>.
Step 1 (Expand):
Possible next word is I.
Hypotheses: [ ("I", P=1.0) ]
Step 2 (Expand):
From "I", we generate two new candidate sequences:
Candidate A: I am. Probability = P(I) * P(am|I) = 1.0 * 0.8 = 0.8
Candidate B: I happy. Probability = P(I) * P(happy|I) = 1.0 * 0.2 = 0.2
Our candidate list is [ ("I am", 0.8), ("I happy", 0.2) ].
Step 2 (Prune):
We sort the candidates by probability: 0.8 > 0.2.
Our beam width is k=2, so we keep both.
Current Beam: Hypothesis 1: ("I am", 0.8), Hypothesis 2: ("I happy", 0.2)
Step 3 (Expand):
Expand Hypothesis 1 ("I am"):
Candidate C: I am Sam. Probability = P("I am") * P(Sam|am) = 0.8 * 0.1 = 0.08
Expand Hypothesis 2 ("I happy"):
Candidate D: I happy am. Probability = P("I happy") * P(am|happy) = 0.2 * 0.9 = 0.18
Our new candidate list is [ ("I am Sam", 0.08), ("I happy am", 0.18) ].
Step 3 (Prune):
We sort the candidates by probability: 0.18 > 0.08.
We keep the top k=2.
Current Beam: Hypothesis 1: ("I happy am", 0.18), Hypothesis 2: ("I am Sam", 0.08)
This is the key moment! The initially less-likely "happy" path has now produced a more probable overall sequence and has been promoted to the top of our beam. The greedy path is now in second place.
Step 4 (Finalize):
Both hypotheses will now be terminated with </s>.
Hypothesis 1 becomes "I happy am </s>" with P=0.18.
Hypothesis 2 becomes "I am Sam </s>" with P=0.08.
The top sequence in the final beam is "I happy am".
Result: Beam Search generates "I happy am" with a probability of 0.18.
Conclusion of the Proof
Greedy Search found the sentence "I am Sam" with a total probability of 0.08.
Beam Search found the sentence "I happy am" with a total probability of 0.18.
Since 0.18 > 0.08, Beam Search found a more probable, and therefore mathematically "better," sentence. It succeeded because it didn't immediately discard the "happy" path just because it was less likely at the first step. By keeping it as a possibility, it discovered that this path ultimately led to a much more likely outcome. This proves the superiority of Beam Search in avoiding local maxima and finding more globally optimal solutions. -->